# MORE EXPERIMENTAL - can we make a movie on the surface using blender? Yes!

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os


In [ ]:
# FOR MY OWN TESTING OF THIS EXAMPLE FOLDER
fs_dir = '/data1/projects/dumoulinlab/Lab_members/Marcus/projects/pilot1/derivatives/freesurfer'
from pfa_scripts.load_saved_info import *
# Movies too!
sub='sub-02'
ts = load_data_tc(sub, 'AS0')['AS0']

### Blender
* requires freesurfer and blender
* a powerful approach and allows for a lot of customization (due to the blender api flexibility, which can be called via python)
* The script will load the inflated and pial mesh, with the option to slide between the 2 (i.e. customize how inflated you want the surface to be)
* You can load several colormaps at once, and flip between them 
* If you are feeling adventurous, you can even create an animation over time, (e.g., plot timecourse info on the surface). This is a bit experimental, and may take up a lot of data and computing power. I haven't fully explored it. 


In [ ]:
from dag_prf_utils.blender_tools import BlendMaker


In [ ]:

# ts = ts[:,:10]
from dag_prf_utils.blender_tools import BlendMaker
bm = BlendMaker(
    sub=sub,
    fs_dir=fs_dir,
    output_dir='z_blend',    
    ow=False, 
)
# This will create a folder with 4 .ply meshes
# -> left-inflated, right-inflated, left-pial, right-pial
# & 2 csv files with rgb information for:
# -> cortical curvature, cortical thickness
from dag_prf_utils.cmap_functions import dag_get_col_vals
# Convert the time series to a movie
b_movie = dag_get_col_vals(ts, cmap='RdBu_r', vmin=-1, vmax=1)
b_movie = b_movie.astype(np.float32)

np.savez_compressed('z_blend/movie.npz', movie=b_movie)
bm.launch_blender()

In [ ]:

bm.launch_blender(
    run_blender=True,
)

In [ ]:
bp_script_add_animation='''
# ADD ANIMATION TO VERTEX COLOR LAYERS...
obj = bpy.data.objects[hemi_list[0]]

total_n_layers = len(obj.data.vertex_colors)
layer_names = list(obj.data.vertex_colors.keys())
def set_vcols(frame):
        
    layer_to_plot = int(total_n_layers * frame / total_n_frames)            

    if layer_to_plot>total_n_layers:
        layer_to_plot = total_n_layers
    for hemi in hemi_list:
        bpy.data.meshes[hemi].attributes.active_color_index = layer_to_plot#layer_names[layer_to_plot]

def my_handler(scene):
    frame = scene.frame_current
    set_vcols(frame)

bpy.app.handlers.frame_change_pre.append(my_handler) 
'''